In [1]:
from src.SimulationEngine import SimulationEngine
from src.RestaurantOptimizer import RestaurantOptimizer 
import json
import os
import time

# Obtener la ruta absoluta al directorio que contiene tu script
dir_path = os.getcwd()

# Usar la ruta absoluta para abrir el archivo config.json
with open(os.path.join(dir_path, 'config.json')) as f:
    config = json.load(f)


In [2]:
def optimize_random_layout():
    start_time = time.time()

    for _ in range(5):
        simulation_engine = SimulationEngine(
            duration=config['simulation_duration'],
            arrival_rate=config['arrival_rate'],
            waiter_amount=config['number_of_waiters']
        )

        optimizer = RestaurantOptimizer(
            simulation_engine=simulation_engine,
            initial_temp=config['initial_temp'],
            final_temp=config['final_temp'],
            alpha=config['alpha'],
            max_iter=config['max_iter'],
            nights_per_layout=config['nights_per_layout'],
            initial_grid=config['optimizer_grid'],
            num_tables=config['number_of_tables'],
            rules_priority=config['rules_priority']
        )

        best_config, best_tips, best_waiting_time, init_tips, init_wait = optimizer.simulated_annealing()
        print("Mejor configuración:")
        for row in best_config:
            print(row)
        print("Mejor promedio de propinas por noche:", best_tips)
        print("Mejor promedio de waiting time por noche:", best_waiting_time)

        end_time = time.time()
        print("Total execution time: {} seconds".format(end_time - start_time), "\n")

        return init_tips, init_wait, best_tips, best_waiting_time

In [ ]:
import scipy.stats as stats

# Datos de ejemplo
initial_setup_times = []  # Tiempos de servicio en minutos
optimized_setup_times = []

initial_tips = []  # Propinas en dólares
optimized_tips = []

for _ in range(5):
    init_tips, init_wait, best_tips, best_waiting_time = optimize_random_layout()
    initial_setup_times.append(init_wait)
    optimized_setup_times.append(best_waiting_time)

    initial_tips.append(init_tips)
    optimized_tips.append(best_tips)

# Realizar el t-test para los tiempos de servicio
service_time_ttest = stats.ttest_ind(initial_setup_times, optimized_setup_times)
print(f"T-test para tiempos de servicio: estadístico={service_time_ttest.statistic}, p-valor={service_time_ttest.pvalue}")

# Realizar el t-test para las propinas
tips_ttest = stats.ttest_ind(initial_tips, optimized_tips)
print(f"T-test para propinas: estadístico={tips_ttest.statistic}, p-valor={tips_ttest.pvalue}")

# Interpretar los resultados
alpha = 0.05
if service_time_ttest.pvalue < alpha:
    print("Rechazamos la hipótesis nula para los tiempos de servicio: la disposición optimizada tiene un efecto significativo.")
else:
    print("No podemos rechazar la hipótesis nula para los tiempos de servicio.")

if tips_ttest.pvalue < alpha:
    print("Rechazamos la hipótesis nula para las propinas: la disposición optimizada tiene un efecto significativo.")
else:
    print("No podemos rechazar la hipótesis nula para las propinas.")